In [ ]:
# django rest app example

# pip install django_rest_framework

# django-admin start restproject

# cd restproject

# python manage.py startapp restapp

# settings.py

# INSTALLED_APPS = [
#      ....
#     'rest_framework'
#     'restapp'
#]


# DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.postgresql',
        'NAME': 'testorm_4',
        'USER': 'testdb',
        'PASSWORD': 'password',
        'HOST': 'localhost',
        'PORT': '5432'
    }
}

# models.py
from django.db import models

# Create your models here.
class Book(models.Model):
    booktitle = models.CharField(max_length=100)  
    bookauthor = models.CharField(max_length=50) 
    class Meta:
        db_table = "books"


# running migration to create tables based on models
# python manage.py makemigrations
# pytthon manage.py migrate


# serializers.py

from dataclasses import fields
from rest_framework import serializers
from .models import Book

class BookSerializer(serializers.ModelSerializer):
    booktitle = serializers.CharField(max_length=50)
    bookauthor = serializers.CharField(max_length=50)
    class Meta:
        model = Book
        fields = ('__all__')
        
# views.py
from django.shortcuts import render
from rest_framework.views import APIView
from rest_framework.response import Response
from rest_framework import status
from .models import Book
from .serializers import BookSerializer

class BookViews(APIView):
    def get(self,request,id=None):
        if id:
            book = Book.objects.get(id=id)
            serializer = BookSerializer(book)
            return Response({"status": "success", "data": serializer.data}, status=status.HTTP_200_OK)
        books = Book.objects.all()
        serializer = BookSerializer(books, many=True)
        return Response({"status": "success","data": serializer.data}, status=status.HTTP_200_OK)

    def post(self,request):
        serializer = BookSerializer(data = request.data)
        if serializer.is_valid():
            serializer.save()
            return Response({"status":"success","data":request.data},status=status.HTTP_200_OK)
        else:
            return Response({"status": "error", "data": serializer.errors}, status=status.HTTP_400_BAD_REQUEST)



# urls.py (application)
from django.urls import path
from . import views

urlpatterns = [
    path('books',views.BookViews.as_view(),name='books')
]


# urls.py (project)
from django.contrib import admin
from django.urls import path, include 

urlpatterns = [
    path('admin/', admin.site.urls),
    path('restapp/',include(('restapp.urls','restapp'),namespace='restapp'))
]